In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/blptask2data/dev_v2.csv')

df.head()

,id,instruction,test_list
0,1,একটি ফাংশন লিখুন যা দিয়ে দেওয়া জোড়া থেকে তৈ...,"""['assert max_chain_length([Pair(5, 24), Pair(..."
1,2,একটি প্রদত্ত স্ট্রিং-এ প্রথম পুনরাবৃত্ত অক্ষর ...,"""['assert first_repeated_char(\""abcabc\"") == \..."
2,3,একটি ফাংশন লিখুন যা n এর চেয়ে ছোট বা সমান একট...,"""['assert get_ludic(10) == [1, 2, 3, 5, 7]', '..."


In [ ]:
import openai

client = openai.OpenAI(api_key="YOUR_API_KEY")

In [ ]:
from tqdm.auto import tqdm
import time

translated_instructions = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Translating"):
    bangla_prompt = row["instruction"]

    prompt = f"""
You are a programming question rewriter.
Your task is to convert the following Bangla programming problem into a clear English programming question.

Bangla problem:
{bangla_prompt}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",   
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512
        )

        english_question = response.choices[0].message.content.strip()

    except Exception as e:
        print(f"⚠️ Error for ID {row['id']}: {e}")
        english_question = "[Translation Failed]"

    translated_instructions.append(english_question)

    time.sleep(2)  # avoid hitting rate limits (50 req/min)


🔁 Translating:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
translated_instructions[:2]

['English version:\n\nWrite a function that finds the length of the longest chain that can be formed from a given list of pairs.\n\nExample function signature:\nmax_chain_length(lst, n)\n\nHere, lst is a list of n pairs. Each pair consists of two integers (a, b), where a < b. A pair (c, d) can follow another pair (a, b) in the chain if b < c. The function should return the maximum length of such a chain that can be formed from the given pairs.',
 'English version:\n\nWrite a Python function to find the first repeated character in a given string.\n\nExample:  \nfirst_repeated_char(s)']

In [7]:
# Remove the prefix if present
cleaned_instructions = [
    s.removeprefix("English version:\n\n") for s in translated_instructions
]

print(cleaned_instructions)

['Write a function that finds the length of the longest chain that can be formed from a given list of pairs.\n\nExample function signature:\nmax_chain_length(lst, n)\n\nHere, lst is a list of n pairs. Each pair consists of two integers (a, b), where a < b. A pair (c, d) can follow another pair (a, b) in the chain if b < c. The function should return the maximum length of such a chain that can be formed from the given pairs.', 'Write a Python function to find the first repeated character in a given string.\n\nExample:  \nfirst_repeated_char(s)', 'Here is the rewritten English programming question:\n\nWrite a function that returns all Ludic numbers less than or equal to n.\n\nExample:  \nget_ludic(n)']


In [ ]:
# Create new DataFrame
out_df = pd.DataFrame({
    "id": df["id"],
    "instruction": cleaned_instructions,
})

# Save to CSV
out_df.to_csv("translated-gpt.csv", index=False, encoding="utf-8")
print("Saved translated_output.csv with id | instruction(translated) ")

✅ Saved translated_output.csv with id | instruction(translated) 


In [9]:
out_df.head()

,id,instruction
0,1,Write a function that finds the length of the ...
1,2,Write a Python function to find the first repe...
2,3,Here is the rewritten English programming ques...
